<a href="https://colab.research.google.com/github/Himanshu-jn52/-poem-or-novel-Classifier-AI-ML/blob/main/poemVSnovel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries.

In [ ]:
import re
import nltk
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **DATA PREPROCESSING**

Collecting required infromation from dataset provided and adding to the dataframe.
Dataframe has two columns:
1.   Content - contents of Poem and Novel
2.   Index - for Poem = 0 and for Novel = 1



In [ ]:
df = pd.DataFrame()
df1 = pd.read_csv("Dataset/all.csv",usecols=["author", "content", "poem name", "age","type"])
df2 = pd.read_csv("Dataset/booksummaries600.txt",delimiter = "\t",header=None)
df2 = df2.drop([0,1], axis = 1)
df2.columns = ["novel name", "author", "date", "type", "content"]

for x in df1["content"][:500]:
    try:
      df = df.append([[x, 0]],ignore_index=True)
    except:
      continue

for x in df2["content"][0:500]:
    try:
      df = df.append([[x, 1]],ignore_index=True)
    except:
      continue

df.columns = ['Content', 'Index']

Cleaning the data 

In [ ]:
cleaned_data = []             #to store cleaned data
for i in range(0, 1000):
  content = re.sub('[^a-zA-Z]', ' ', df['Content'][i])    #to replace all character & punctuation with white space
  content = content.lower()        
  content = content.split()
  ps = PorterStemmer()   # Creating an stem object
  all_stopwords = stopwords.words('english')   # collecting all the stopwords in english
  content = [ps.stem(word) for word in content if not word in set(all_stopwords)] # removing stopwords from contents
  content = ' '.join(content)
  cleaned_data.append(content)

Converting contents into binary format (Creating BOW)

In [ ]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(cleaned_data).toarray() #fit and transform to vector
y = df.iloc[:, -1].values


# **MODELLING & TRAINING**

Generating train & test data with test size of 20%.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

Defining two methods of Naive - Bayes

1.  Gaussian Naive Bayes
2.  Multinomial Naive Bayes




In [ ]:
#Gaussian Naive Bayes
Gclassifier = GaussianNB() 
Gclassifier.fit(X_train, y_train)

#Multinomial Naive Bayes
Mclassifier = MultinomialNB()
Mclassifier.fit(X_train, y_train)

print(Gclassifier.score(X_test,y_test)) #---> 0.905
print(Mclassifier.score(X_test,y_test)) #---> 0.98 will proceed with Multinomial Naive Bayes

0.905
0.98


Performing prediction for Multinomial Naive Bayes 

In [ ]:
y_pred = Mclassifier.predict(X_test)
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

Will evaluate the modela and check the accuracy score.

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 94   4]
 [  0 102]]


0.98

In [ ]:

# dump trained model to pickle file
pickle.dump(Mclassifier, open("model.pkl", "wb"))

In [51]:
#calling main function

def getType(content, loaded_model):
  content = re.sub('[^a-zA-Z]', ' ', content)
  content = content.lower()
  content = content.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  content = [ps.stem(word) for word in content if not word in set(all_stopwords)]
  content = ' '.join(content)
  corpus = [content]
  #cv = CountVectorizer(max_features = 1500)
  new_X_test = cv.transform(corpus).toarray()
  new_y_pred = loaded_model.predict(new_X_test)
  return new_y_pred

if __name__=="__main__":
  loaded_model = pickle.load(open("model.pkl", "rb"))
  output = getType(str(input("Enter content...")), loaded_model)
  if output[0]==1:
    print("Novel")
  else :
    print("Poem")

Enter content...Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random "ultra-violence." Alex's friends ("droogs" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and Pete, who mostly plays along as the droogs indulge their taste for ultra-violence. Characterized as a sociopath and a hardened juvenile delinquent, Alex is also intelligent and quick-witted, with sophisticated taste in music, being particularly fond of Beethoven, or "Lovely Ludwig Van." The novel begins with the droogs sitting in their favorite hangout (the Korova Milkbar), drinking milk-drug cocktails, called "milk-plus", to hype themselves for the night's mayhem. They assault a scholar walking home from the public library, rob a store leaving the owner and his wife bloodied and unconscious, stomp a panhandling derelict, then scuffle with a rival gang. Joyriding through the coun